In [1]:
import cv2
import numpy as np


def iou_batch(bb_dt, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_dt = np.expand_dims(bb_dt, 1)

    xx1 = np.maximum(bb_dt[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_dt[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_dt[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_dt[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_dt[..., 2] - bb_dt[..., 0]) * (bb_dt[..., 3] - bb_dt[..., 1])                                      
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
    return(o)  


def giou_batch(bb_dt, bb_gt):
    """
    From SORT: Computes GIoU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_dt = np.expand_dims(bb_dt, 1)

    xx1 = np.maximum(bb_dt[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_dt[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_dt[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_dt[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    I = w * h
    area1 = (bb_dt[..., 2] - bb_dt[..., 0]) * (bb_dt[..., 3] - bb_dt[..., 1])
    area2 = (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
    U = area1 + area2 - I
    IoU = I / U

    C_xx1 = np.minimum(bb_dt[..., 0], bb_gt[..., 0])
    C_yy1 = np.minimum(bb_dt[..., 1], bb_gt[..., 1])
    C_xx2 = np.maximum(bb_dt[..., 2], bb_gt[..., 2])
    C_yy2 = np.maximum(bb_dt[..., 3], bb_gt[..., 3])
    C_w = C_xx2 - C_xx1
    C_h = C_yy2 - C_yy1
    C_area = C_w * C_h
    GIoU = IoU - (C_area - U)/C_area

    return GIoU 

In [2]:
def draw_bboxs(bbox1, bbox2, iou, giou):
    img = 255 * np.ones((500, 500, 3), dtype=np.uint8)
    img = cv2.rectangle(img, (bbox1[0], bbox1[1]), (bbox1[2], bbox1[3]), (255, 0, 0), 1)
    img = cv2.rectangle(img, (bbox2[0], bbox2[1]), (bbox2[2], bbox2[3]), (0, 0, 255), 1)
    
    img = cv2.putText(img, " iou=%.2f"%iou, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
    img = cv2.putText(img, "giou=%.2f"%giou, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
    return img

In [7]:
import random

def random_bboxs(bboxes1):
    bboxes2 = bboxes1.copy()
    x1_gain = random.randint(-40,40)
    y1_gain = random.randint(-120,120)
    x2_gain = random.randint(-5,5)
    y2_gain = random.randint(-5,5)
    
    bboxes2[0,0] += x1_gain
    bboxes2[0,1] += y1_gain
    bboxes2[0,2] = bboxes2[0,2] + x1_gain + x2_gain
    bboxes2[0,3] = bboxes2[0,3] + y1_gain + y2_gain
    
    return bboxes2

In [8]:
bboxes1 = np.array(
    [
        [100, 200, 150, 300],
    ]
)

for ii in range(1000):
    bboxes2 = random_bboxs(bboxes1)
    iou = iou_batch(bboxes1, bboxes2)[0,0]
    giou = giou_batch(bboxes1, bboxes2)[0,0]
    img = draw_bboxs(bboxes1[0], bboxes2[0], iou, giou)

    cv2.imwrite("res/%d.jpg"%ii, img)

3